In [3]:
# Read json file
import json
import numpy as np
import pandas as pd
import datetime as dt
from pandas.io.json import json_normalize
np.set_printoptions(precision=4, suppress=True)

In [4]:
with open('data0928/forecasts.json') as json_file:
    fcast_data = json.load(json_file)

# Key check
print(fcast_data.keys())#, ifp_data.keys())
fcast_data = fcast_data['predictions'] #'predictions' is the only first tier key of the json file
#fcast_data = fcast_data['forecasts'] #'predictions' is the only first tier key of the json file

# Length check
print(len(fcast_data))#, len(ifp_data))

fcasts = json_normalize(fcast_data)

#turk = turk.sort_value(by="user_id")
#print(np.min(turk['user_id'].values), np.max(turk['user_id'].values))
#turk
fcasts = fcasts.sort_values(by=['question_id','user_id','created_at'], ascending=[True, True, True])
fcasts.to_csv('data0928/fcasts0928.csv')
fcasts.iloc[0]["forecast"]
#fcasts["team_id"].unique
fcasts.columns
#fcasts

dict_keys(['predictions'])
347992


Index(['auto', 'confidence', 'created_at', 'downvotes', 'forecast',
       'question_id', 'rationale', 'team_id', 'upvotes', 'user_id', 'wager'],
      dtype='object')

In [1]:
pre_qid = -1
pre_uid = -1
pre_created_at = -1
pre_wager = -1
cnt = 0
#timestamps = np.zeros(len(fcasts))
timestamp = fcasts['created_at'].values
for idx, row in fcasts.iterrows():
    # if a user updated a prediction but not the wager, use the previous wager for this prediction
    if row['question_id']==pre_qid and row['user_id']==pre_uid and pre_created_at!=row['created_at']:
        fcasts.loc[idx,'wager'] = pre_wager
    pre_qid = row['question_id']
    pre_uid = row['user_id']
    pre_created_at = row['created_at']
    pre_wager = row['wager']
    # create the timestamp for the record
    ### 0982 version misses "wagered_at" attribute
    #if type(row['wagered_at'])==str:
    #    timestamp[cnt] = row['wagered_at']
    cnt += 1
    if cnt%10000==0: print(cnt)

fcasts['timestamp'] = timestamp
fcasts.to_csv('data0928/fcasts0928_refined.csv')
fcasts

NameError: name 'fcasts' is not defined

In [3]:
with open('data0928/ifps.json') as json_file:
    ifp_data = json.load(json_file)
ifp_data = ifp_data['ifps'] #'ifps' is the only first tier key of the json file
print(ifp_data[0].keys())

dict_keys(['site_id', 'binary?', 'discover_question_id', 'published_at', 'id', 'active?', 'featured', 'type', 'prediction_sets_count', 'description', 'predictors_count', 'brier_score', 'answers', 'updated_at', 'ends_at', 'resolved_at', 'resolution_notes', 'name', 'created_at', 'membership_id', 'resolved?', 'void_reason', 'comments_count', 'starts_at', 'use_ordinal_scoring', 'answers_count'])


In [9]:
# Create ifp_map from question_id to other information
# Processing resolved ifp
ifps = json_normalize(ifp_data)
print('total ifps:', len(ifps))
#print('resolved ifps:', sum(ifps['resolved?']))
#ifps = ifps[ifps['resolved?']]

MAX_NUM_OF_OUTCOME = np.max(ifps['answers_count'].values)
if MAX_NUM_OF_OUTCOME<=1: MAX_NUM_OF_OUTCOME = 2
print('MAX_NUM_OF_OUTCOME:', MAX_NUM_OF_OUTCOME)
print('total valid ifps:', len(ifps))
#print(ifp_data[0])

ifps = ifps.rename(columns={'created_at':'ifp_created_at'})
n = 0
type_column, truth_column, open_date_column, close_date_column, resolve_date_column = [], [], [], [], []
for idx, row in ifps.iterrows():
    # find out the ground truth outcome for current IFP
    if not row['resolved?']: outcome = np.NAN
    elif row['answers_count']<=1:
        outcome = 2 - int(row['answers'][0]['probability']>0.01)
    else:
        for i, ans in enumerate(row['answers']):
            if ans['probability']>0.01:
                outcome = i + 1
                break
    truth_column.append(outcome)
    answers_count = max(row['answers_count'],2) # Binary question has a 1 in attribute 'answers_count'

    # find out close date and resolved date of an IFP
    print(row['ends_at'], row['resolved_at'], row['answers'][0]['correctness_known_at'])
    close_date = row['ends_at'][:10]
    resolved_date = '3000-01-01' # default as resolved date not known
    tmp_date = row['resolved_at']
    if tmp_date is not None: 
        if tmp_date[:10]<close_date: close_date = tmp_date[:10]
        if tmp_date[:10]<resolved_date: resolved_date = tmp_date[:10]
    tmp_date = row['answers'][0]['correctness_known_at']
    if tmp_date is not None: 
        if tmp_date[:10]<close_date: close_date = tmp_date[:10]
        if tmp_date[:10]<resolved_date: resolved_date = tmp_date[:10]

    type_column.append(answers_count)
    open_date_column.append(row['ifp_created_at'][:10])
    close_date_column.append(close_date[:10])
    resolve_date_column.append(resolved_date[:10])
    n += 1
truth = np.array(truth_column)
print(truth)
ifps['map_id'] = np.arange(n)
ifps['ground_truth'] = np.array(truth_column)
ifps['type'] = np.array(type_column)
ifps['ifp_opens_at'] = np.array(open_date_column)
ifps['ifp_closes_at'] = np.array(close_date_column)
ifps['ifp_resolves_at'] = np.array(resolve_date_column)
ifps.to_csv('data0928/ifps0928_refined.csv')
ifps

total ifps: 191
MAX_NUM_OF_OUTCOME: 5
total valid ifps: 191
2018-08-31T11:01:59 2018-09-04T11:49:06.911000 2018-09-04T08:22:47
2018-04-25T11:01:55 2018-04-27T11:28:23.490000 2018-04-26T14:00:11
2018-09-07T11:01:54 2018-09-10T09:36:04.619000 2018-09-10T09:30:02
2018-09-07T11:01:53 2018-09-10T10:01:00.314000 2018-09-10T09:55:43
2018-04-30T11:01:50 2018-06-05T12:14:33.503000 2018-06-05T12:13:29
2018-04-10T11:01:49 2018-04-18T10:26:28.821000 2018-03-16T20:59:44
2018-08-31T11:01:49 2018-09-04T11:48:09.006000 2018-09-04T08:22:11
2018-03-29T11:01:17 2018-04-10T12:53:33.544000 2018-03-30T20:59:39
2018-04-30T11:01:15 2018-05-16T10:47:59.899000 2018-05-15T10:43:19
2018-08-31T11:01:09 2018-09-04T12:30:10.094000 2018-08-31T21:00:27
2018-05-31T11:01:04 2018-06-07T15:22:41.590000 2018-06-07T12:00:59
2018-09-07T11:01:04 2018-09-10T09:57:41.805000 2018-09-10T09:40:41
2018-06-30T11:01:59 2018-07-05T10:09:50.196000 2018-07-05T09:52:46
2018-03-31T11:01:56 2018-04-30T10:33:48.056000 2018-04-30T10:32:17
20

,active?,answers,answers_count,binary?,brier_score,comments_count,ifp_created_at,description,discover_question_id,ends_at,...,starts_at,type,updated_at,use_ordinal_scoring,void_reason,map_id,ground_truth,ifp_opens_at,ifp_closes_at,ifp_resolves_at
0,False,"[{'ended?': True, 'probability': 0.0, 'questio...",1,True,0.0,0,2018-03-14T09:30:14.627000,A “significant leadership disruption” includes...,210,2018-08-31T11:01:59,...,2018-03-14T09:30:59,2,2018-09-04T20:52:04.321000,False,None,0,2.0,2018-03-14,2018-08-31,2018-09-04
1,False,"[{'ended?': True, 'probability': 0.0, 'questio...",5,False,0.0,0,2018-03-14T09:30:08.108000,This question will be resolved using the last ...,189,2018-04-25T11:01:55,...,2018-03-14T09:30:55,5,2018-07-17T13:27:39.727000,True,None,1,3.0,2018-03-14,2018-04-25,2018-04-26
2,False,"[{'ended?': True, 'probability': 0.0, 'questio...",1,True,0.0,0,2018-03-14T09:30:22.205000,Article 7.1 enables the Council of the Europea...,215,2018-09-07T11:01:54,...,2018-03-14T09:30:54,2,2018-09-10T10:49:22.093000,False,None,2,2.0,2018-03-14,2018-09-07,2018-09-10
3,False,"[{'ended?': True, 'probability': 0.0, 'questio...",1,True,0.0,0,2018-03-14T09:30:20.682000,Temporary lifting of the blockade or partial l...,214,2018-09-07T11:01:53,...,2018-03-14T09:30:53,2,2018-09-10T11:47:25.284000,False,None,3,2.0,2018-03-14,2018-09-07,2018-09-10
4,False,"[{'ended?': True, 'probability': 1.0, 'questio...",3,False,0.0,0,2018-03-14T09:30:09.912000,This question will be resolved using monthly d...,207,2018-04-30T11:01:50,...,2018-03-14T09:30:50,3,2018-07-17T13:27:39.821000,True,None,4,1.0,2018-03-14,2018-04-30,2018-06-05
5,False,"[{'ended?': True, 'probability': 0.0, 'questio...",5,False,0.0,0,2018-03-14T09:30:13.460000,his question will be resolved using the Multis...,209,2018-04-10T11:01:49,...,2018-03-14T09:30:49,5,2018-07-17T13:27:40.039000,True,None,5,4.0,2018-03-14,2018-03-16,2018-03-16
6,False,"[{'ended?': True, 'probability': 0.0, 'questio...",1,True,0.0,0,2018-03-14T09:30:15.609000,A “significant leadership disruption” includes...,211,2018-08-31T11:01:49,...,2018-03-14T09:30:49,2,2018-09-04T19:47:52.990000,False,None,6,2.0,2018-03-14,2018-08-31,2018-09-04
7,False,"[{'ended?': True, 'probability': 0.0, 'questio...",5,False,0.0,0,2018-03-14T09:30:24.415000,This question will be resolved using the closi...,204,2018-03-29T11:01:17,...,2018-03-14T09:30:17,5,2018-07-17T13:27:40.796000,True,None,7,4.0,2018-03-14,2018-03-29,2018-03-30
8,False,"[{'ended?': True, 'probability': 0.0, 'questio...",3,False,0.0,0,2018-03-14T09:30:11.902000,EMPRES Global Animal Disease Information Syste...,208,2018-04-30T11:01:15,...,2018-03-14T09:30:15,3,2018-07-17T13:27:39.903000,True,None,8,2.0,2018-03-14,2018-04-30,2018-05-15
9,False,"[{'ended?': True, 'probability': 1.0, 'questio...",5,False,0.0,0,2018-03-14T09:30:25.004000,"Saudi Arabia, the UAE, Bahrain, and Egypt seve...",217,2018-08-31T11:01:09,...,2018-03-14T09:30:09,5,2018-09-05T08:29:55.047000,True,None,9,1.0,2018-03-14,2018-08-31,2018-08-31


In [11]:
# Merge all fcasts records with ifp and turk user information
with open('data0928/turk_users.json') as json_file:
    turk_data = json.load(json_file)
turks = json_normalize(turk_data)

fcasts_all = pd.read_csv('data0928/fcasts0928_refined.csv')
print('# of fcasts:', len(fcasts_all))
merged = fcasts_all.merge(right=turks, how="left", on="user_id")
merged = merged.fillna(value={'turk_user': False})
print('# of fcasts after merging turk identification:', len(merged))
merged = merged.merge(right=ifps[['id', 'resolved?', 'type', 'ground_truth',
                                  'answers', 'ifp_opens_at', 'ifp_closes_at', 'ifp_resolves_at']], 
                      how="inner", left_on="question_id", right_on="id")
print('# of fcasts after merging ifp information:', len(merged))
print(merged.columns)
merged.to_csv('data0928/fcasts0928_with_ifp_turk_info.csv')

/Users/heliaguin/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# of fcasts: 347992
# of fcasts after merging turk identification: 347992
# of fcasts after merging ifp information: 347992
Index(['Unnamed: 0', 'auto', 'confidence', 'created_at', 'downvotes',
       'forecast', 'question_id', 'rationale', 'team_id', 'upvotes', 'user_id',
       'wager', 'timestamp', 'turk_user', 'id', 'resolved?', 'type',
       'ground_truth', 'answers', 'ifp_opens_at', 'ifp_closes_at',
       'ifp_resolves_at'],
      dtype='object')
